In [1]:
from nlinec.data.load import load_data, get_all_types
from nlinec.data.preprocessing import get_granularity

In [4]:
data = load_data('augmented_train.json', explode=True)

793487it [00:11, 67290.47it/s] 


In [7]:
data['granularity'] = data['full_type'].apply(get_granularity)

In [8]:
data

,full_type,mention_span,sentence,granularity
0,/other,the history and science of aviation and spacef...,"Located in Washington, D.C., United States, it...",1
1,/other,his debut,"The seventh child of the Jackson family, he ma...",1
2,/location/country,We,We did not do anything at that time.,2
3,/location,We,We did not do anything at that time.,1
4,/other,antibody,`` We don't know the effect of our antibody on...,1
...,...,...,...,...
1865003,/other/art/film,his films,"As a boy, he donated his Bar Mitzvah money to ...",3
1865004,/other,his films,"As a boy, he donated his Bar Mitzvah money to ...",1
1865005,/other/art,his films,"As a boy, he donated his Bar Mitzvah money to ...",2
1865006,/other/art,"Japanese martial arts classes such as Aikido ,...",Sensei is often used to address the teacher in...,2


In [2]:
gran_types = []
for i in [1, 2, 3]:
    gran_types.append(get_all_types(granularity=i))

251039it [00:03, 69644.45it/s]


TypeError: unhashable type: 'list'